### Cleaning dataset

In [11]:
import pandas as pd
import numpy as np

import yaml

In [12]:
try:
    with open('../config.yaml') as file:
        config = yaml.safe_load(file)
except:
    print("Yaml file not found.")

In [13]:
# Load dataset q1
q1_df = pd.read_csv(config['input_data']['q1_dataset'])
q1_df.head()

,username,country,country_code,question_no,uid,year,answer,value_na,value
0,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2000,Mixed system [0.5],0.5,0.5
1,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2001,Mixed system [0.5],0.5,0.5
2,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2002,Mixed system [0.5],0.5,0.5
3,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2003,Mixed system [0.5],0.5,0.5
4,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2004,Mixed system [0.5],0.5,0.5


In [4]:
# Load dataset q2
q2_df = pd.read_csv(config['input_data']['q2_dataset'])
q2_df.head()

,username,country,country_code,question_no,uid,year,answer,value_na,value
0,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2000,Unequal div. with predom. influence of the leg...,0.67,0.67
1,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2001,Unequal div. with predom. influence of the leg...,0.67,0.67
2,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2002,Unequal div. with predom. influence of the leg...,0.67,0.67
3,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2003,Unequal div. with predom. influence of the leg...,0.67,0.67
4,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2004,Unequal div. with predom. influence of the leg...,0.67,0.67


In [14]:
def cleaning_judicial_autonomy_data(qx_df):

    # Step 1: Create copy of dataframe
    df = qx_df.copy()
    
    # Step 2: Adjust dataframe shape
    # Pivot long dataframe to wide dataframe
    df_pivoted = df.pivot(index=['username', 'country', 'country_code', 'year'], 
                          columns=['uid'], 
                          values=['value'])
    
    # Reset index and flattening multi-level column names
    df_pivoted.reset_index(inplace=True)
    df_pivoted.columns = [col[1] if col[1] else col[0] for col in df_pivoted.columns]

    # Step 3: Clean country and user names
    # Remove usernames
    remove_usernames = ('ADMIN123', 
                        'ALB22uFF4m',
                        'BEL22cEw8t', 
                        'BIH22q2nOU', 
                        'DNK22KFh1N', 
                        'MNE22N8NJv', 
                        'NLD22Ba53p', 
                        'SRB22L4wbh')
    df_countries_cleaned = df_pivoted[~df_pivoted['username'].isin(remove_usernames)]

    # Replace country names
    df_countries_cleaned.loc[:,'country'] = df_countries_cleaned['country'].replace({
        'Czech Republic': 'Czechia',
        'Republic of Albania': 'Albania',
        'Republic of Serbia': 'Serbia',
        'Bosnia and Herzegovina (BiH)': 'Bosnia and Herzegovina',
        'Montenegro (MON)': 'Montenegro',
        'Kingdom of Belgium': 'Belgium'})
    
    # Step 4: Remove subjective questions
    # Drop columns that contain '_subj'
    df_cleaned = df_countries_cleaned.drop(list(df_countries_cleaned.filter(regex='_subj')), axis=1)

    return df_cleaned

In [15]:
q1_cleaned = cleaning_judicial_autonomy_data(q1_df)
#q1_cleaned.to_csv(config['output_data']['q1_dataset'])
q1_cleaned    

,username,country,country_code,year,q1c1_apjuac,q1c1_apjufc,q1c1_apjuhc,q1c1_appealac,q1c1_appealfc,q1c1_appealhc,...,q1c3_sanctscale,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_regbudget,q1c4_sameright,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
46,ALB33wGG5n,Albania,ALA,2000,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
47,ALB33wGG5n,Albania,ALA,2001,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
48,ALB33wGG5n,Albania,ALA,2002,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
49,ALB33wGG5n,Albania,ALA,2003,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
50,ALB33wGG5n,Albania,ALA,2004,0.5,0.5,0.0,0.0,0.0,0.0,...,0.5,1.0,0.00,1.0,0.0,1.0,0.0,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,UKR22Gf5Kx,Ukraine,UKR,2018,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1100,UKR22Gf5Kx,Ukraine,UKR,2019,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1101,UKR22Gf5Kx,Ukraine,UKR,2020,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1102,UKR22Gf5Kx,Ukraine,UKR,2021,0.5,0.5,0.5,1.0,1.0,1.0,...,1.0,0.5,0.67,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [8]:
q2_cleaned = cleaning_judicial_autonomy_data(q2_df)
#q2_cleaned.to_csv(config['output_data']['q2_dataset'])
q2_cleaned  

C:\Users\Friederike\AppData\Local\Temp\ipykernel_4208\1456239474.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned = df_countries_cleaned.drop(list(df_countries_cleaned.filter(regex='_subj')), axis=1, inplace=True, )


### Long version cleaning function

In [4]:
# Pivot long dataframe to wide dataframe
q1_pivoted = q1_df.pivot(index=['username', 'country', 'country_code', 'year'], 
                      columns=['uid'], 
                      values=['value'])

# Reset index and flattening multi-level column names
# Use lambda function to access tuple storing the multi-level column names
q1_pivoted.reset_index(inplace=True)
q1_pivoted.columns = [col[1] if col[1] else col[0] for col in q1_pivoted.columns]
#q1_pivoted

In [5]:
# Filter out countries / users
remove_usernames = ('ADMIN123', 'ALB22uFF4m','BEL22cEw8t', 'BIH22q2nOU', 'DNK22KFh1N', 'MNE22N8NJv', 'NLD22Ba53p', 'SRB22L4wbh')
q1_countries_cleaned = q1_pivoted[~q1_pivoted['username'].isin(remove_usernames)]
#q1_countries_cleaned

In [6]:
# Replace country names
q1_countries_cleaned.loc[:,'country'] = q1_countries_cleaned['country'].replace({
    "Czech Republic": "Czechia",
    "Republic of Albania": "Albania",
    "Republic of Serbia": "Serbia",
    "Bosnia and Herzegovina (BiH)": "Bosnia and Herzegovina",
    "Montenegro (MON)": "Montenegro",
    "Kingdom of Belgium": "Belgium"})
#q1_countries_cleaned

In [7]:
# Drop columns that contain '_subj' (i.e. subjective answers)
q1_subj_cleaned = q1_countries_cleaned.drop(list(q1_countries_cleaned.filter(regex='_subj')), axis=1)
#q1_subj_cleaned

### Create subsets

In [24]:
def subset_dataset_into_indicators(qx_cleaned):

    subsets = {

    # Actors involved in appointment procedures of judges
    'q1_appointment_actors': qx_cleaned[['country',
                                        'year',
                                        'q1c1_apjufc',
                                        'q1c1_apjuac',
                                        'q1c1_apjuhc',
                                        'q1c1_appresidfc',
                                        'q1c1_appresidac',
                                        'q1c1_appresidhc',
                                        'q1c1_exvetofc',
                                        'q1c1_exvetoac',
                                        'q1c1_exvetohc']],
    
    # Selection and appointment criteria for judges
    'q1_appointment_criteria': qx_cleaned[['country',
                                          'year',
                                          'q1c1_critfclaw', 
                                          'q1c1_critaclaw',
                                          'q1c1_crithclaw', 
                                          'q1c1_critfcints',
                                          'q1c1_critacints', 
                                          'q1c1_crithcints',
                                          'q1c1_probju']],

    # Transparency and mechanisms for appeal in appointment procedures of judges
    'q1_appointment_transparency': qx_cleaned[['country',
                                              'year',
                                              'q1c1_transplaw', 
                                              'q1c1_appealfc',
                                              'q1c1_appealac',
                                              'q1c1_appealhc']],
    # Tenure and term in office of judges
    'q1_judge_tenure': qx_cleaned[['country',
                                  'year',
                                  'q1c2_termfcju', 
                                  'q1c2_termacju',
                                  'q1c2_termpresid', 
                                  'q1c2_termhcju',
                                  'q1c2_retireage']],
    
    # Immunity and non-transferability of judges
    'q1_judge_immunity': qx_cleaned[['country',
                                    'year',
                                    'q1c2_juabsimmun', 
                                    'q1c2_jufuncimmun',
                                    'q1c2_juremove', 
                                    'q1c2_jutransf']],

    # Salaries and bonuses of judges
    'q1_judge_salary': qx_cleaned[['country',
                                  'year',
                                  'q1c2_jusalary', 
                                  'q1c2_jubonus',
                                  'q1c2_jupension']],
        
    # Disciplinary proceedings against judges
    'q1_disciplinary_proceedings': qx_cleaned[['country',
                                              'year',
                                              'q1c3_fairtrial', 
                                              'q1c3_disciplaw',
                                              'q1c3_discipints', 
                                              'q1c3_discipbody',
                                              'q1c3_initdiscip', 
                                              'q1c3_decdiscip',
                                              'q1c3_immunlift', 
                                              'q1c3_sanctscale',
                                              'q1c3_appealdiscip']],
        
    # Conflict of interest, recusal from cases and evaluation of judges
    'q1_conflict_recusal_evaluation': qx_cleaned[['country',
                                          'year',
                                          'q1c3_judisclos', 
                                          'q1c3_jurestrict',
                                          'q1c3_jurecuse']],
                                          
    # Composition and functioning of the judicial self-governing bodies
    'q1_judicial_self_governance': qx_cleaned[['country',
                                             'year',
                                             'q1c4_whocharge', 
                                             'q1c4_whoselect',
                                             'q1c4_whochair', 
                                             'q1c4_competence',
                                             'q1c4_sameright', 
                                             'q1c4_reasondecis']], 
    
    # Administration, functioning and budget of courts                                         
    'q1_courts_administration': qx_cleaned[['country',
                                           'year',
                                           'q1c4_casealloc', 
                                           'q1c4_regbudget',
                                           'q1c4_manbudget']]
    }

    return subsets
    

In [26]:
subset_indicators = subset_dataset_into_indicators(q1_cleaned)

q1_appointment_actors = subset_indicators['q1_appointment_actors']
q1_appointment_criteria = subset_indicators['q1_appointment_criteria']
q1_appointment_transparency = subset_indicators['q1_appointment_transparency']
q1_judge_tenure = subset_indicators['q1_judge_tenure']
q1_judge_immunity = subset_indicators['q1_judge_immunity']
q1_judge_salary = subset_indicators['q1_judge_salary']
q1_disciplinary_proceedings = subset_indicators['q1_disciplinary_proceedings']
q1_conflict_recusal_evaluation = subset_indicators['q1_conflict_recusal_evaluation']
q1_judicial_self_governance = subset_indicators['q1_judicial_self_governance']
q1_courts_administration = subset_indicators['q1_courts_administration']


In [27]:
def filling_nan_with_mode(subsets):
    for subset in subsets:
        subset.dropna(subset=['number'])

In [29]:
filling_nan_with_mode(subset_indicators)

AttributeError: 'str' object has no attribute 'dropna'

### Check for missing values in each subset

In [35]:
q1_courts_administration.isna().sum().sort_values(ascending=False)

q1c4_casealloc    29
q1c4_manbudget    23
country            0
year               0
q1c4_regbudget     0
dtype: int64

In [20]:
cols_with_nan = q1_courts_administration.columns[q1_courts_administration.isna().any()].tolist()
missing_counts = q1_courts_administration.groupby('country')[cols_with_nan].apply(lambda x: x.isna().sum()).reset_index()
#numeric_cols = missing_counts.select_dtypes(exclude='object').columns
missing_counts

,country,q1c4_casealloc,q1c4_manbudget
0,Albania,0,0
1,Armenia,18,0
2,Austria,0,0
3,Azerbaijan,0,0
4,Belgium,0,0
5,Bosnia and Herzegovina,11,0
6,Bulgaria,0,0
7,Croatia,0,0
8,Cyprus,0,0
9,Czechia,0,0


In [22]:
missing_counts = q1_courts_administration.groupby('country').apply(lambda x: x.isna().sum()).reset_index()
missing_counts

C:\Users\Friederike\AppData\Local\Temp\ipykernel_15840\2583031799.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_counts = q1_courts_administration.groupby('country').apply(lambda x: x.isna().sum()).reset_index()


ValueError: cannot insert country, already exists

### Check and deal with missing values

In [38]:

cols_with_nan = q1_cleaned.columns[q1_cleaned.isna().any()].tolist()
missing_counts = q1_cleaned.groupby('country')[cols_with_nan].apply(lambda x: x.isna().sum()).reset_index()
numeric_cols = missing_counts.select_dtypes(exclude='object').columns

# Summing up NaN counts across these columns for each row to calculate the 'total_nan'
missing_counts['total_nan'] = missing_counts[numeric_cols].sum(axis=1)
sorted_missing_counts = missing_counts.sort_values(by='total_nan', ascending=False)
sorted_missing_counts

#missing_counts['total_nan'] = missing_counts.dtypes(exclude='object').sum(axis=1)
#missing_counts.dtypes

,country,q1c1_apjuac,q1c1_apjufc,q1c1_appealac,q1c1_appresidac,q1c1_appresidfc,q1c1_appresidhc,q1c1_critacints,q1c1_critaclaw,q1c1_exvetoac,...,q1c3_initdiscip,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_sameright,q1c4_whochair,q1c4_whocharge,q1c4_whoselect,total_nan
17,Iceland,18,0,18,18,0,0,18,18,18,...,0,0,0,0,0,0,0,0,0,218
18,Ireland,0,0,0,0,0,0,0,0,0,...,0,0,19,0,19,19,19,19,19,205
9,Czechia,0,0,0,0,0,0,0,0,0,...,0,0,23,0,23,23,23,0,0,161
39,United Kingdom,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,114
20,Latvia,0,0,0,0,0,0,0,0,0,...,0,0,10,0,10,10,10,10,10,106
14,Germany,0,0,0,0,0,2,0,0,0,...,21,0,0,23,0,23,23,0,0,94
36,Switzerland,0,23,0,0,23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,90
19,Italy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,23,0,0,89
5,Bosnia and Herzegovina,0,0,0,0,0,0,0,0,23,...,0,11,0,0,0,0,0,2,0,86
10,Estonia,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,2,2,2,2,74


In [9]:
# Check datatypes and missing values
q1_subj_cleaned.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 920 entries, 46 to 1103
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   username           920 non-null    object 
 1   country            920 non-null    object 
 2   country_code       920 non-null    object 
 3   year               920 non-null    int64  
 4   q1c1_apjuac        902 non-null    float64
 5   q1c1_apjufc        897 non-null    float64
 6   q1c1_apjuhc        920 non-null    float64
 7   q1c1_appealac      902 non-null    float64
 8   q1c1_appealfc      920 non-null    float64
 9   q1c1_appealhc      920 non-null    float64
 10  q1c1_appresidac    902 non-null    float64
 11  q1c1_appresidfc    897 non-null    float64
 12  q1c1_appresidhc    918 non-null    float64
 13  q1c1_critacints    902 non-null    float64
 14  q1c1_critaclaw     902 non-null    float64
 15  q1c1_critfcints    920 non-null    float64
 16  q1c1_critfclaw     920 non-nu

In [10]:
q1_subj_cleaned.isna().sum().sort_values(ascending=False)

q1c2_retireage       783
q1c2_jubonus         471
q1c3_immunlift       288
q1c3_evalints        233
q1c4_whochair        183
q1c4_sameright        82
q1c4_competence       56
q1c4_reasondecis      56
q1c1_exvetoac         41
q1c4_whocharge        33
q1c4_whoselect        31
q1c4_casealloc        29
q1c1_apjufc           23
q1c1_appresidfc       23
q1c4_manbudget        23
q1c2_jutransf         23
q1c3_initdiscip       21
q1c2_termacju         18
q1c1_appresidac       18
q1c1_apjuac           18
q1c1_appealac         18
q1c1_critacints       18
q1c1_critaclaw        18
q1c1_appresidhc        2
q1c1_transplaw         2
q1c1_appealhc          0
q1c2_juabsimmun        0
q1c1_probju            0
q1c1_exvetofc          0
q1c1_exvetohc          0
q1c1_crithclaw         0
q1c1_critfcints        0
q1c1_critfclaw         0
q1c1_crithcints        0
country_code           0
username               0
country                0
year                   0
q1c1_appealfc          0
q1c1_apjuhc            0


In [11]:
q1_subj_cleaned.isna().mean().sort_values(ascending=False)*100

q1c2_retireage       85.108696
q1c2_jubonus         51.195652
q1c3_immunlift       31.304348
q1c3_evalints        25.326087
q1c4_whochair        19.891304
q1c4_sameright        8.913043
q1c4_competence       6.086957
q1c4_reasondecis      6.086957
q1c1_exvetoac         4.456522
q1c4_whocharge        3.586957
q1c4_whoselect        3.369565
q1c4_casealloc        3.152174
q1c1_apjufc           2.500000
q1c1_appresidfc       2.500000
q1c4_manbudget        2.500000
q1c2_jutransf         2.500000
q1c3_initdiscip       2.282609
q1c2_termacju         1.956522
q1c1_appresidac       1.956522
q1c1_apjuac           1.956522
q1c1_appealac         1.956522
q1c1_critacints       1.956522
q1c1_critaclaw        1.956522
q1c1_appresidhc       0.217391
q1c1_transplaw        0.217391
q1c1_appealhc         0.000000
q1c2_juabsimmun       0.000000
q1c1_probju           0.000000
q1c1_exvetofc         0.000000
q1c1_exvetohc         0.000000
q1c1_crithclaw        0.000000
q1c1_critfcints       0.000000
q1c1_cri

In [12]:
print(f"Number of columns that contain at least one missing value: {q1_subj_cleaned.isnull().any(axis = 0).sum()}")
print(f"Number of rows that contain at least one missing value: {q1_subj_cleaned.isnull().any(axis = 1).sum()}")

Number of columns that contain at least one missing value: 25
Number of rows that contain at least one missing value: 875


In [13]:
q1_subj_cleaned.loc[:, q1_subj_cleaned.isna().any()]
#df2.loc[:, df2.isnull().any()])

,q1c1_apjuac,q1c1_apjufc,q1c1_appealac,q1c1_appresidac,q1c1_appresidfc,q1c1_appresidhc,q1c1_critacints,q1c1_critaclaw,q1c1_exvetoac,q1c1_transplaw,...,q1c3_immunlift,q1c3_initdiscip,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_sameright,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
46,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,1.0,0.33,...,1.0,0.0,1.0,0.00,1.0,0.0,0.0,NaN,0.5,0.5
47,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,1.0,0.33,...,1.0,0.0,1.0,0.00,1.0,0.0,0.0,NaN,0.5,0.5
48,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,1.0,0.33,...,1.0,0.0,1.0,0.00,1.0,0.0,0.0,NaN,0.5,0.5
49,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,1.0,0.33,...,1.0,0.0,1.0,0.00,1.0,0.0,0.0,NaN,0.5,0.5
50,0.5,0.5,0.0,0.5,0.5,0.0,1.0,0.0,1.0,0.33,...,1.0,0.0,1.0,0.00,1.0,0.0,0.0,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.00,...,1.0,1.0,0.5,0.67,0.0,0.0,1.0,1.0,1.0,1.0
1100,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.00,...,1.0,1.0,0.5,0.67,0.0,0.0,1.0,1.0,1.0,1.0
1101,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.00,...,1.0,1.0,0.5,0.67,0.0,0.0,1.0,1.0,1.0,1.0
1102,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.00,...,1.0,1.0,0.5,0.67,0.0,0.0,1.0,1.0,1.0,1.0


In [14]:
q1c4_nan = q1_subj_cleaned[q1_subj_cleaned['q1c4_manbudget'].isna()]
q1c4_nan

,username,country,country_code,year,q1c1_apjuac,q1c1_apjufc,q1c1_apjuhc,q1c1_appealac,q1c1_appealfc,q1c1_appealhc,...,q1c3_sanctscale,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_regbudget,q1c4_sameright,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
322,DEU22vd5fP,Germany,DEU,2000,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
323,DEU22vd5fP,Germany,DEU,2001,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
324,DEU22vd5fP,Germany,DEU,2002,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
325,DEU22vd5fP,Germany,DEU,2003,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
326,DEU22vd5fP,Germany,DEU,2004,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
327,DEU22vd5fP,Germany,DEU,2005,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
328,DEU22vd5fP,Germany,DEU,2006,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
329,DEU22vd5fP,Germany,DEU,2007,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,0.0,NaN,NaN,0.5,1.0
330,DEU22vd5fP,Germany,DEU,2008,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0
331,DEU22vd5fP,Germany,DEU,2009,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,0.5,1.0


q1c2_retireage       783
q1c2_jubonus         471
q1c3_immunlift       288
q1c3_evalints        233
q1c4_whochair        183
q1c4_sameright        82
q1c4_competence       56
q1c4_reasondecis      56
q1c1_exvetoac         41
q1c4_whocharge        33
q1c4_whoselect        31
q1c4_casealloc        29
q1c1_apjufc           23
q1c1_appresidfc       23
q1c4_manbudget        23
q1c2_jutransf         23
q1c3_initdiscip       21
q1c2_termacju         18
q1c1_appresidac       18
q1c1_apjuac           18
q1c1_appealac         18
q1c1_critacints       18
q1c1_critaclaw        18
q1c1_appresidhc        2
q1c1_transplaw         2